In [ ]:
import sys
sys.path.append('tools')
import os
import re
import uuid
import json
import time
from collections import Counter

from IPython.core.display import HTML
import numpy as np
import pandas as pd
from sympy import sympify, Eq, simplify
from latex2sympy2 import latex2sympy

import parse_data
import api

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
cwd = os.getcwd()
res_dir = os.path.join(cwd, 'result')

papers = ['2024-1', '2024-2']

models = os.listdir(res_dir)
models.remove(".DS_Store")

# Inference Results

In [4]:
results = {}

for model in models:

    model_res_dir = os.path.join(res_dir, model)
    fpaths = parse_data.find_files_with_suffix(model_res_dir, '.jsonl')
    results[model] = {}
    total_tokens, infer_time = 0, 0

    for fpath in fpaths:
        content = parse_data.read_jsonl(fpath)
        for i in content:
            if not i['token_usage']:
                continue

            total_tokens += i['token_usage']['total_tokens']
            # gpt use batch api
            if "gpt" in model:
                infer_time = 2136
            else:
                infer_time += i['infer_time(s)']
            

    infer_speed = round(total_tokens/infer_time, 2)

    results[model]['Total_tokens'] = total_tokens
    results[model]['Infer_time(s)'] = infer_time
    results[model]['Infer_speed(tokens/s)'] = infer_speed

# Recreate DataFrame with numeric values for processing and displaying without formatting
df = pd.DataFrame(results).T

# Sort the DataFrame by the Average column
df.sort_values(by='Infer_speed(tokens/s)', ascending=False, inplace=True)

# Function to highlight the top three values in each column
def highlight_top_three(s):
    is_numeric = pd.to_numeric(s, errors='coerce').notnull().all()
    if is_numeric:
        sorted_s = s.sort_values(ascending=False)
        colors = ['color: blue', 'color: green', 'color: pink']
        top3 = sorted_s.head(3)
        highlight_colors = [''] * len(s)
        for i, v in enumerate(s):
            if v in top3.values:
                highlight_colors[i] = colors[top3.values.tolist().index(v)]
        return highlight_colors
    return ['' for _ in s]

# Apply the highlighting function
styled_df = df.style.apply(highlight_top_three, subset=['Total_tokens', 'Infer_time(s)', 'Infer_speed(tokens/s)']).set_caption("Model Inference Results")

# Format the values as percentages with two decimal places
styled_df = styled_df.format({
    'Total_tokens': '{:,.2f}',
    'Infer_time(s)': '{:,.2f}',
    'Infer_speed(tokens/s)': '{:,.2f}'
})

styled_df
print("""Note:
- 因为数据管理的失误导致 abab6.5s-chat 和 yi-large 的 total_otkens 部分丢失，
      所以这两个的 total_tokens 最低的。
- 因为 gpt4o 使用了 batch API 进行推理，所以推理速度高出了两个量级。
""")

,Total_tokens,Infer_time(s),Infer_speed(tokens/s)
gpt-4o-2024-05-13,"2,432,776.00","2,136.00","1,138.94"
abab6.5s-chat,"1,693,468.00","24,510.42",69.09
doubao-pro-32k-240615,"1,780,180.00","35,288.62",50.45
mistral-large-2402,"2,255,747.00","45,889.72",49.16
glm-4-0520,"2,675,433.00","57,578.30",46.47
qwen2-72b-instruct,"2,299,737.00","61,043.24",37.67
moonshot-v1-8k,"2,701,808.00","77,959.69",34.66
ERNIE-4.0-Turbo-8K,"2,122,628.00","63,461.00",33.45
llama3-70b-instruct,"1,994,395.00","61,382.33",32.49
deepseek-coder,"2,606,396.00","82,008.87",31.78


Note:
- 因为数据管理的失误导致 abab6.5s-chat 和 yi-large 的 total_otkens 部分丢失，
      所以这两个的 total_tokens 最低的。
- 因为 gpt4o 使用了 batch API 进行推理，所以推理速度高出了两个量级。



# Instruction-following

## Response Structure Format

+ **Motivation**: To investigate whether the model strictly adheres to the specified response format given in the system prompt or at the beginning of the user prompt.
+ **Explanation**:
  + The system prompt typically consists of two or three parts: "### Original Question" (Optional), "### Solution Process", and "### Answer". Additionally, there is always a question index at the beginning, such as "## 1".
  + There are two modes in the evaluation: strict or lax. Strict mode means concatenating `\n` to the titles as mentioned in the system prompt, such as "### Original Question\n", while lax mode only includes the text.
  + If a model response includes all essential parts, that response follows the instruction and returns `True`; otherwise, it returns `False`.
  + The types of listing bad cases are missing a single part, missing all parts ("ALL"), and missing a combination of parts ("Others").
  + The table is sorted by the `("instruction-following", "True")`.

In [5]:
def validate_content_format(content_text: str, format_settings: dict, validation_mode: str) -> tuple:
    """
    Validate the content text against predefined format patterns based on language and the inclusion of a question section.
    Supports different validation modes ('strict' or 'lax') to adjust the pattern matching criteria.

    Args:
    content_text (str): The text to be validated.
    format_settings (dict): Dictionary specifying 'lang' (language) and 'print_question' (bool for including the question section).
    validation_mode (str): Mode of validation ('strict' or 'lax') to determine pattern matching criteria.

    Returns:
    tuple: A tuple where the first element is a boolean indicating if all required patterns are found,
           and the second element is a tuple of missing patterns, if any.
    """
    # Patterns based on language and whether the question is included in the format
    format_patterns = {
        "zh": {
            True: ["^## \d{,2}", "### 原题", "### 答案", "### 解题过程"],
            False: ["^## \d{,2}", "### 答案", "### 解题过程"]
        },
        "en": {
            True: ["^## \d{,2}", "### Original Problem", "### Answer", "### Solution Process"],
            False: ["^## \d{,2}", "### Answer", "### Solution Process"]
        }
    }

    # Apply stricter criteria based on the validation mode
    if validation_mode == 'strict':
        for language in format_patterns:
            for question_presence in [True, False]:
                format_patterns[language][question_presence] = [pattern + "\n" for pattern in format_patterns[language][question_presence]]

    # Retrieve the relevant patterns based on language and question inclusion setting
    required_patterns = format_patterns[format_settings['lang']][format_settings['print_question']]

    patterns_found = True
    missing_patterns = []

    # Evaluate each pattern against the content text
    for pattern in required_patterns:
        compiled_pattern = re.compile(pattern)
        if not re.search(compiled_pattern, content_text):
            missing_patterns.append(pattern)
            patterns_found = False

    # Sort missing patterns for consistent output
    sorted_missing_patterns = sorted(missing_patterns)

    if set(sorted_missing_patterns) == set(required_patterns):
        sorted_missing_patterns.append('ALL')

    return patterns_found, tuple(sorted_missing_patterns)

def process_results(models, papers, res_dir, validation_mode):
    """
    Processes the results of different models based on their response formatting.

    Args:
        models (list): List of model names to process.
        papers (list): List of papers (currently unused in this function).
        res_dir (str): Directory where the model result files are located.
        validation_mode (str): Mode of validation, can be 'strict' or another mode.

    Returns:
        pd.DataFrame: A DataFrame containing the evaluation results for each model.
    """
    results = []

    # Iterate over each model
    for model_name in models[:]:
        model_res_dir = os.path.join(res_dir, model_name)
        fpaths = parse_data.find_files_with_suffix(model_res_dir, '.jsonl')

        # Initialize counters for existence and type
        res = {
            'exist': Counter(),
            'type': Counter()
        }

        # Process each file
        for fpath in fpaths:
            content = parse_data.read_jsonl(fpath)

            # Validate content format
            for entry in content:
                result, prompt_parameter = entry['result'], entry['prompt_parameter']
                r1, r2 = validate_content_format(result, prompt_parameter, validation_mode)

                # if not r1:
                #     print("sdafsd", r2)
                #     print(result)
                
                res['exist'][r1] += 1
                if not r1:
                    res["type"][r2] += 1

        # Calculate true, false counts and rate
        true_count = res['exist'][True]
        false_count = res['exist'][False]
        total = true_count + false_count
        rate = f"{true_count / total:.2%}" if total != 0 else "0.00%"

        # Define patterns to aggregate bad case details
        patterns = [
            "^## \d{,2}", "### 原题", "### Original Problem", "### 答案", 
            "### Answer", "### 解题过程", "### Solution Process"
        ]
        if validation_mode == 'strict':
            patterns = [pattern + '\n' for pattern in patterns]
        
        bad_cases_details = {pattern: res['type'][(pattern, )] for pattern in patterns}
        bad_cases_details['ALL'] = sum(res['type'][i] for i in res['type'] if i[-1] == 'ALL')
        bad_cases_details['Others'] = false_count - sum(bad_cases_details.values())

        # Collect model data
        model_data = [true_count, false_count, rate] + list(bad_cases_details.values())
        results.append((model_name, model_data))

    # Define columns based on validation mode
    if validation_mode == 'strict':
        columns = pd.MultiIndex.from_tuples([
            ('Instruction Following', 'True'),
            ('Instruction Following', 'False'),
            ('Instruction Following', 'Rate'),
            ('Bad Cases', r"^## \d{,2}\n"),
            ('Bad Cases', r"### 原题\n"),
            ('Bad Cases', r"### Original Problem\n"),
            ('Bad Cases', r"### 答案\n"),
            ('Bad Cases', r"### Answer\n"),
            ('Bad Cases', r"### 解题过程\n"),
            ('Bad Cases', r"### Solution Process\n"),
            ('Bad Cases', 'ALL'),
            ('Bad Cases', 'Others')
        ])
    else:
        columns = pd.MultiIndex.from_tuples([
            ('Instruction Following', 'True'),
            ('Instruction Following', 'False'),
            ('Instruction Following', 'Rate'),
            ('Bad Cases', "^## \d{,2}"),
            ('Bad Cases', "### 原题"),
            ('Bad Cases', "### Original Problem"),
            ('Bad Cases', "### 答案"),
            ('Bad Cases', "### Answer"),
            ('Bad Cases', "### 解题过程"),
            ('Bad Cases', "### Solution Process"),
            ('Bad Cases', 'ALL'),
            ('Bad Cases', 'Others')
        ])

    # Create DataFrame and sort by 'True' count in descending order
    df = pd.DataFrame([data for _, data in results], index=[model for model, _ in results], columns=columns)
    df.index.name = 'Model'
    df.sort_values(by=('Instruction Following', 'True'), ascending=False, inplace=True)
    df = df.style.set_caption(f'Instruction-Following Degree in Response Structure Formatting: {validation_mode} mode')

    return df

df = process_results(models, papers, res_dir, 'strict')
df

df = process_results(models, papers, res_dir, 'lax')
df

## Answer Format

+ **Motivation**: To investigate whether the model strictly adheres to the specified answer format given in the system prompt or at the beginning of the user prompt.
+ **Required Format**: 
  - **Multiple Choice**: `{'{question number}': [{answers}]}`, for example, `{'99': ['E']}` or `{'98': ['E', 'F']}`.
  - **Fill-in-the-Blank**: `{'{question number}': [{LaTex format answers}]}`, for example, `{'97': ['$0$']}` or `{'96': ['$0$', '$1$']}`.
+ **Explanation**:
  + Investigated three types:
    + The required format: `{'{ques_idx}': \['.+'\]}`
    + Allowed single quotes and double quotes: `{['\"]?{ques_idx}['\"]?: \[['\"]?.+['\"]?\]}`
    + Allowed `\n\s\t`: `{\n*\s*\t*['\"]?{ques_idx}['\"]?: \[['\"]?.+\n*\s*\t*}`
  + The table is sorted by the average column.
  + Mark the top 3 values for each column as "blue", "green", and "pink" respectively.

In [6]:
results = []

# Iterate through each model
for model_name in models:
    model_res_dir = os.path.join(res_dir, model_name)

    # Initialize counters for each pattern
    n1, n2, n3 = 0, 0, 0
    
    # Iterate through each paper
    for paper in papers:
        paper_res_dir = os.path.join(model_res_dir, paper)
        
        # Get all .jsonl files
        fnames = [i for i in os.listdir(paper_res_dir) if i.endswith('.jsonl')]
        
        # Process each file
        for fname in fnames:
            fpath = os.path.join(paper_res_dir, fname)
            content = parse_data.read_jsonl(fpath)

            # Iterate through each item in the file
            for i in range(len(content)):
                std_answer = content[i]['question']['answer']
                ques_idx = content[i]['question']['idx']
                res = content[i]['result']

                # Check the result against three different patterns
                pattern_1 = "{" + f"'{ques_idx}': \['.+'\]" + "}"
                if not re.search(re.compile(pattern_1), res):
                    n1 += 1

                pattern_2 = "{" + f"['\"]?{ques_idx}['\"]?: \[['\"]?.+['\"]?\]" + "}"
                if not re.search(re.compile(pattern_2), res):
                    n2 += 1

                pattern_3 = "{\n*\s*\t*" + f"['\"]?{ques_idx}['\"]?" + ": \[['\"]?.+\n*\s*\t*}"
                if not re.search(re.compile(pattern_3), res):
                    n3 += 1

    # Calculate accuracy for each pattern and the average
    accuracy_1 = (2240 - n1) / 2240
    accuracy_2 = (2240 - n2) / 2240
    accuracy_3 = (2240 - n3) / 2240
    avg_accuracy = (accuracy_1 + accuracy_2 + accuracy_3) / 3
    
    # Append the results for the current model
    results.append((model_name, [f"{accuracy_1:8.2%}", f"{accuracy_2:8.2%}", f"{accuracy_3:8.2%}", f"{avg_accuracy:8.2%}"]))

# Define column names
columns = pd.Index(["{'{ques_idx}': \['.+'\]}", r"{['\"]?{ques_idx}['\"]?: \[['\"]?.+['\"]?\]}", r"{\n*\s*\t*['\"]?{ques_idx}['\"]?: \[['\"]?.+\n*\s*\t*}", 'Average'])

# Create DataFrame
df = pd.DataFrame([data for _, data in results], index=[model for model, _ in results], columns=columns)
df.index.name = 'Model'

# Sort the DataFrame by the Average column
df.sort_values(by='Average', ascending=False, inplace=True)

# Function to highlight the top three values in each column
def highlight_top_three(s):
    sorted_values = s.str.rstrip('%').astype('float').sort_values(ascending=False)
    max_val, second_val, third_val = sorted_values.iloc[0], sorted_values.iloc[1], sorted_values.iloc[2]
    
    def color_font(val):
        val_float = float(val.rstrip('%'))
        if val_float == max_val:
            return 'color: blue'
        elif val_float == second_val:
            return 'color: green'
        elif val_float == third_val:
            return 'color: pink'
        return ''

    return [color_font(v) for v in s]

# Apply the highlighting function and set caption
df = df.style.apply(highlight_top_three, subset=columns).set_caption('Instruction-Following Degree in Answer Formatting')

# Display the styled DataFrame
df

,{'{ques_idx}': \['.+'\]},"{['\""]?{ques_idx}['\""]?: \[['\""]?.+['\""]?\]}","{\n*\s*\t*['\""]?{ques_idx}['\""]?: \[['\""]?.+\n*\s*\t*}",Average
Model,,,,
claude-3-5-sonnet-20240620,99.73%,99.73%,100.00%,99.82%
gemini-1.5-pro-001,98.93%,98.93%,99.64%,99.17%
llama3-70b-instruct,98.35%,99.06%,99.55%,98.99%
doubao-pro-32k-240615,97.63%,98.35%,99.91%,98.63%
qwen2-72b-instruct,97.28%,98.62%,99.73%,98.54%
glm-4-0520,96.88%,97.19%,98.53%,97.53%
mistral-large-2402,96.65%,96.88%,98.48%,97.34%
moonshot-v1-8k,95.22%,97.68%,99.06%,97.32%
gpt-4o-2024-05-13,95.49%,95.71%,97.99%,96.40%


## Reponse Structure & Answer Format

+ Gather the two together to compare the models.

In [7]:
# Define the original dictionaries
# response structure: strict
dict1 = {
    'gpt-4o-2024-05-13': '97.19%',
    'moonshot-v1-8k': '96.96%',
    'claude-3-5-sonnet-20240620': '92.90%',
    'qwen-max': '91.21%',
    'deepseek-chat': '90.98%',
    'deepseek-coder': '89.46%',
    'llama3-70b-instruct': '88.71%',
    'qwen2-72b-instruct': '84.78%',
    'mistral-large-2402': '83.08%',
    'glm-4-0520': '82.37%',
    'doubao-pro-32k-240615': '78.66%',
    'ERNIE-4.0-Turbo-8K': '77.14%',
    'abab6.5s-chat': '68.12%',
    'gemini-1.5-pro-001': '56.25%',
    'yi-large': '33.93%',
    'baichuan4': '13.98%'
}

# response structure: lax
dict2 = {
    'gemini-1.5-pro-001': '100.00%',
    'gpt-4o-2024-05-13': '99.46%',
    'moonshot-v1-8k': '97.10%',
    'claude-3-5-sonnet-20240620': '92.90%',
    'glm-4-0520': '92.77%',
    'deepseek-chat': '92.72%',
    'qwen-max': '91.38%',
    'deepseek-coder': '89.46%',
    'llama3-70b-instruct': '89.06%',
    'qwen2-72b-instruct': '85.31%',
    'mistral-large-2402': '83.84%',
    'yi-large': '81.29%',
    'doubao-pro-32k-240615': '78.66%',
    'ERNIE-4.0-Turbo-8K': '78.08%',
    'abab6.5s-chat': '68.57%',
    'baichuan4': '13.98%'
}

# answer format
dict3 = {
    'claude-3-5-sonnet-20240620': '99.82%',
    'gemini-1.5-pro-001': '99.17%',
    'llama3-70b-instruct': '98.99%',
    'doubao-pro-32k-240615': '98.63%',
    'qwen2-72b-instruct': '98.54%',
    'glm-4-0520': '97.68%',
    'mistral-large-2402': '97.34%',
    'moonshot-v1-8k': '97.32%',
    'gpt-4o-2024-05-13': '96.40%',
    'deepseek-chat': '95.39%',
    'abab6.5s-chat': '93.38%',
    'yi-large': '91.32%',
    'qwen-max': '90.25%',
    'baichuan4': '81.46%',
    'deepseek-coder': '70.64%',
    'ERNIE-4.0-Turbo-8K': '67.07%'
}

# Calculate the average values for dict1 and dict2
avg_dict = {}
for model in dict1:
    if model in dict2:
        value1 = float(dict1[model].strip('%'))
        value2 = float(dict2[model].strip('%'))
        average_value = (value1 + value2) / 2
        avg_dict[model] = average_value

# Integrate avg_dict and dict3 into a single DataFrame
models = sorted(set(avg_dict.keys()).union(dict3.keys()))
data = {
    'Response Structure': [avg_dict.get(model, 'N/A') for model in models],
    'Answer': [float(dict3.get(model, 'N/A').strip('%')) for model in models]
}

# Create the DataFrame
df = pd.DataFrame(data, index=models)
df.index.name = 'Model'

# Calculate the average of the two columns and add it as a new column
df['Average'] = df.mean(axis=1)

# Sort the DataFrame by the Average column
df.sort_values(by='Average', ascending=False, inplace=True)

# Function to highlight the top three values in each column
def highlight_top_three(s):
    is_numeric = pd.to_numeric(s, errors='coerce').notnull().all()
    if is_numeric:
        sorted_s = s.sort_values(ascending=False)
        colors = ['color: blue', 'color: green', 'color: pink']
        top3 = sorted_s.head(3)
        highlight_colors = [''] * len(s)
        for i, v in enumerate(s):
            if v in top3.values:
                highlight_colors[i] = colors[top3.values.tolist().index(v)]
        return highlight_colors
    return ['' for _ in s]

# Apply the highlighting function
styled_df = df.style.apply(highlight_top_three, subset=['Response Structure', 'Answer', 'Average'])

# Format the values as percentages with two decimal places
styled_df = styled_df.format({
    'Response Structure': '{:.2f}%',
    'Answer': '{:.2f}%',
    'Average': '{:.2f}%'
})

# Set the caption for the DataFrame
styled_df = styled_df.set_caption('Instruction-following in Formatting')

# Display the styled DataFrame
styled_df


,Response Structure,Answer,Average
Model,,,
gpt-4o-2024-05-13,98.32%,96.40%,97.36%
moonshot-v1-8k,97.03%,97.32%,97.17%
claude-3-5-sonnet-20240620,92.90%,99.82%,96.36%
llama3-70b-instruct,88.88%,98.99%,93.94%
deepseek-chat,91.85%,95.39%,93.62%
glm-4-0520,87.57%,97.68%,92.62%
qwen2-72b-instruct,85.05%,98.54%,91.79%
qwen-max,91.29%,90.25%,90.77%
mistral-large-2402,83.46%,97.34%,90.40%


# Answer Accuracy

## Question Index

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            "question_idx":{
                "accuracy": ..,
                "avg_score": ..., 
                "total_score": ..., 
                "correct_cnt": ..., 
                "total_cnt": ...
            }
        }
    }
}
```

In [10]:
# Define a function to set background colors for specified row name and its data cells
def highlight_row_name(df, row_name):
    # Function to highlight data cells of the specified row
    def highlight_avg(row):
        if row.name == row_name:
            return ['background-color: yellow']*len(row)
        else:
            return ['']*len(row)
    
    # Apply the cell style
    styled = df.apply(highlight_avg, axis=1)
    # Highlight the specified row name
    styled.set_table_styles({
        row_name: [{'selector': 'th', 'props': 'background-color: yellow;'}],
    }, overwrite=False, axis=1)
    return styled

def generate_acc_table(ans_data, papers, paper_type):

    tables = []

    all_data = []

    for paper in papers:
        data = []
        for model_name, questions in ans_data[paper].items():
            for question_idx, metrics in questions.items():
                question_num = int(question_idx.replace('question', ''))
                data.append([model_name, question_num, metrics["accuracy"]])

        df = pd.DataFrame(data, columns=["Model", "Question Index", "Accuracy"])

        # Pivot table to get the desired format
        pivot_df = df.pivot(index="Model", columns="Question Index", values="Accuracy")

        # Calculate the average accuracy for each model and add as a new column
        pivot_df['Avg'] = pivot_df.mean(axis=1)

        # Sort by Avg column in descending order
        pivot_df = pivot_df.sort_values(by='Avg', ascending=False)

        column_avg = pivot_df.mean()
        pivot_df.loc['Avg'] = column_avg  # 在排序之后添加平均值行

        # Apply the highlighting function
        styled_df = pivot_df.style.apply(highlight_top_three, subset=[i for i in range(1, 15)] + ['Avg'])

        # Set the caption for the DataFrame
        styled_df = styled_df.set_caption(f'{paper} Question Accuracy: {paper_type}')

        styled_df = styled_df.format('{:.2%}')

        # Display the styled DataFrame
        tables.append(styled_df)
        
        # Collect model-avg values for each paper
        for model_name, avg in pivot_df['Avg'].items():
            all_data.append([model_name, paper, avg])

    # Construct final DataFrame for model-avg values
    final_df = pd.DataFrame(all_data, columns=["Model", "Paper", "Avg"])

    # Pivot the final DataFrame to get the desired format
    final_pivot_df = final_df.pivot(index="Model", columns="Paper", values="Avg")

    # Calculate the average accuracy for each model and add as a new column
    final_pivot_df['Avg'] = final_pivot_df.mean(axis=1)
    final_pivot_df = final_pivot_df.sort_values(by='Avg', ascending=False, axis=0)

    # Apply the highlighting function
    styled_df = final_pivot_df.style.apply(highlight_top_three, subset=[i for i in papers] + ['Avg'])
    styled_df = highlight_row_name(styled_df, 'Avg')
    # Format the final DataFrame
    styled_df = styled_df.format('{:.2%}')

    # Set the caption for the final DataFrame
    styled_df = styled_df.set_caption(f'Average Accuracy for {paper_type}')

    # Display the final styled DataFrame
    tables.append(styled_df)

    return tables

def display_multi_table(table_list):
    ''' 
    Accepts a list of pandas DataFrame objects. Displays the first n-1 DataFrames in one vertical column,
    and the last DataFrame in a separate column alongside, with improved visual distinction.
    '''
    if not table_list:
        return None

    # 为 df1 和 df2 的排列添加边框和背景色，以便在视觉上区分
    vertical_tables = '<table style="border-spacing: 10px 0;">' + \
                      ''.join(['<tr><td style="background-color: #f0f0f0; border: 1px solid #ddd;">' + df.to_html() + '</td></tr>' for df in table_list[:-1]]) + \
                      '</table>'
    
    # 最后一个 DataFrame 单独在一个单元格中
    horizontal_table = '<td>' + table_list[-1].to_html() + '</td>'

    # 构造最终的表格布局
    final_layout = '<table><tr><td>' + vertical_tables + '</td>' + horizontal_table + '</tr></table>'

    return HTML(final_layout)

In [11]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, total_score = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                total_cnt += 1
                if i['answer']['correct']:
                    correct_cnt += 1
                total_score += i['answer']['score']

            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['total_score'] = total_score
            ans_data[paper][model][ques_idx]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][ques_idx]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_acc_table(ans_data, papers, 'ALL')
display_multi_table(tables)

Question Index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Avg
Model,,,,,,,,,,,,,,,
deepseek-coder,98.75%,32.50%,50.00%,52.50%,91.25%,76.25%,100.00%,63.75%,65.00%,23.75%,23.75%,28.75%,45.00%,1.25%,53.75%
qwen2-72b-instruct,98.75%,57.50%,58.75%,23.75%,97.50%,95.00%,86.25%,56.25%,70.00%,7.50%,36.25%,5.00%,18.75%,20.00%,52.23%
deepseek-chat,100.00%,27.50%,50.00%,52.50%,98.75%,86.25%,100.00%,38.75%,60.00%,28.75%,21.25%,22.50%,35.00%,2.50%,51.70%
gemini-1.5-pro-001,38.75%,42.50%,95.00%,26.25%,100.00%,77.50%,90.00%,62.50%,63.75%,25.00%,0.00%,7.50%,32.50%,2.50%,47.41%
gpt-4o-2024-05-13,97.50%,63.75%,56.25%,20.00%,65.00%,57.50%,92.50%,3.75%,42.50%,1.25%,56.25%,15.00%,36.25%,7.50%,43.93%
claude-3-5-sonnet-20240620,43.75%,93.75%,81.25%,28.75%,75.00%,95.00%,56.25%,18.75%,42.50%,13.75%,0.00%,10.00%,26.25%,3.75%,42.05%
baichuan4,57.50%,52.50%,70.00%,33.75%,60.00%,70.00%,67.50%,40.00%,38.75%,3.75%,13.75%,0.00%,13.75%,35.00%,39.73%
yi-large,56.25%,60.00%,75.00%,32.50%,62.50%,52.50%,67.50%,42.50%,45.00%,2.50%,26.25%,0.00%,18.75%,13.75%,39.64%
ERNIE-4.0-Turbo-8K,43.75%,36.25%,73.75%,25.00%,53.75%,88.75%,63.75%,17.50%,17.50%,3.75%,31.25%,3.75%,11.25%,0.00%,33.57%


In [12]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, total_score = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                if i['prompt_parameter']['lang'] == 'en':
                    total_cnt += 1
                    if i['answer']['correct']:
                        correct_cnt += 1
                    total_score += i['answer']['score']

            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['total_score'] = total_score
            ans_data[paper][model][ques_idx]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][ques_idx]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_acc_table(ans_data, papers, 'EN')
display_multi_table(tables)

Question Index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Avg
Model,,,,,,,,,,,,,,,
qwen2-72b-instruct,100.00%,32.50%,67.50%,25.00%,100.00%,92.50%,77.50%,77.50%,70.00%,5.00%,57.50%,7.50%,0.00%,17.50%,52.14%
deepseek-chat,100.00%,12.50%,50.00%,45.00%,100.00%,72.50%,100.00%,32.50%,45.00%,15.00%,30.00%,5.00%,30.00%,5.00%,45.89%
gemini-1.5-pro-001,37.50%,45.00%,92.50%,27.50%,100.00%,70.00%,85.00%,65.00%,50.00%,25.00%,0.00%,2.50%,32.50%,2.50%,45.36%
deepseek-coder,100.00%,20.00%,50.00%,47.50%,90.00%,57.50%,100.00%,42.50%,32.50%,2.50%,17.50%,22.50%,50.00%,2.50%,45.36%
gpt-4o-2024-05-13,100.00%,55.00%,55.00%,25.00%,72.50%,55.00%,97.50%,0.00%,40.00%,0.00%,60.00%,20.00%,35.00%,10.00%,44.64%
claude-3-5-sonnet-20240620,37.50%,95.00%,90.00%,25.00%,75.00%,100.00%,27.50%,17.50%,57.50%,20.00%,0.00%,2.50%,27.50%,0.00%,41.07%
baichuan4,65.00%,52.50%,72.50%,30.00%,85.00%,72.50%,47.50%,32.50%,42.50%,0.00%,15.00%,0.00%,15.00%,37.50%,40.54%
yi-large,50.00%,60.00%,75.00%,32.50%,57.50%,42.50%,55.00%,35.00%,55.00%,5.00%,30.00%,0.00%,15.00%,10.00%,37.32%
ERNIE-4.0-Turbo-8K,40.00%,35.00%,57.50%,22.50%,57.50%,90.00%,82.50%,22.50%,25.00%,7.50%,35.00%,2.50%,15.00%,0.00%,35.18%


In [13]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, total_score = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                if i['prompt_parameter']['lang'] == 'zh':
                    total_cnt += 1
                    if i['answer']['correct']:
                        correct_cnt += 1
                    total_score += i['answer']['score']

            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['total_score'] = total_score
            ans_data[paper][model][ques_idx]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][ques_idx]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_acc_table(ans_data, papers, 'ZH')
display_multi_table(tables)

Question Index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Avg
Model,,,,,,,,,,,,,,,
deepseek-coder,97.50%,45.00%,50.00%,57.50%,92.50%,95.00%,100.00%,85.00%,97.50%,45.00%,30.00%,35.00%,40.00%,0.00%,62.14%
deepseek-chat,100.00%,42.50%,50.00%,60.00%,97.50%,100.00%,100.00%,45.00%,75.00%,42.50%,12.50%,40.00%,40.00%,0.00%,57.50%
qwen2-72b-instruct,97.50%,82.50%,50.00%,22.50%,95.00%,97.50%,95.00%,35.00%,70.00%,10.00%,15.00%,2.50%,37.50%,22.50%,52.32%
gemini-1.5-pro-001,40.00%,40.00%,97.50%,25.00%,100.00%,85.00%,95.00%,60.00%,77.50%,25.00%,0.00%,12.50%,32.50%,2.50%,49.46%
gpt-4o-2024-05-13,95.00%,72.50%,57.50%,15.00%,57.50%,60.00%,87.50%,7.50%,45.00%,2.50%,52.50%,10.00%,37.50%,5.00%,43.21%
claude-3-5-sonnet-20240620,50.00%,92.50%,72.50%,32.50%,75.00%,90.00%,85.00%,20.00%,27.50%,7.50%,0.00%,17.50%,25.00%,7.50%,43.04%
yi-large,62.50%,60.00%,75.00%,32.50%,67.50%,62.50%,80.00%,50.00%,35.00%,0.00%,22.50%,0.00%,22.50%,17.50%,41.96%
baichuan4,50.00%,52.50%,67.50%,37.50%,35.00%,67.50%,87.50%,47.50%,35.00%,7.50%,12.50%,0.00%,12.50%,32.50%,38.93%
doubao-pro-32k-240615,45.00%,47.50%,75.00%,32.50%,55.00%,95.00%,97.50%,2.50%,5.00%,2.50%,0.00%,45.00%,20.00%,0.00%,37.32%


## Temperature

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            "temperature":{
                "accuracy": ..,
                "avg_score": ..., 
                "total_score": ..., 
                "correct_cnt": ..., 
                "total_cnt": ...
            }
        }
    }
}
```

temperature values: [0.1, 0.3, 0.5, 0.7, 0.9]

In [14]:
values = [0.1, 0.3, 0.5, 0.7, 0.9]

In [15]:
def generate_table(ans_data, papers, paper_type):

    def highlight_max_in_row(df):
        styles = pd.DataFrame('', index=df.index, columns=df.columns)
        for row in df.index:  
            # ignore the Total/Average column
            max_col = df.loc[row][:-1].idxmax()
            styles.loc[row, max_col] = 'text-decoration: underline'
        return styles

    tables = []

    # 用于收集所有模型在所有paper中的精度数据
    all_data = []

    for paper in papers:
        data = []
        for model_name, questions in ans_data[paper].items():
            for temp_value, metrics in questions.items():
                question_num = round(temp_value, 2)
                data.append([model_name, question_num, metrics["accuracy"]])

        df = pd.DataFrame(data, columns=["Model", "Temperature Value", "Accuracy"])
        pivot_df = df.pivot(index="Model", columns="Temperature Value", values="Accuracy")
        pivot_df['Avg'] = pivot_df.mean(axis=1)
        pivot_df = pivot_df.sort_values(by='Avg', ascending=False)
        pivot_df.loc['Avg'] = pivot_df.mean()
        styled_df = pivot_df.style.apply(highlight_max_in_row, axis=None).apply(highlight_top_three, subset=pivot_df.columns.drop('Avg').tolist() + ['Avg'])
        styled_df = styled_df.set_caption(f'{paper} Question Accuracy: {paper_type}')
        styled_df = styled_df.format('{:.2%}')
        tables.append(styled_df)
        
        # 为最终的汇总表收集数据
        for index, row in pivot_df.iterrows():
            for col in pivot_df.columns.drop('Avg'):
                all_data.append([index, col, row[col]])

    # 构建一个包含所有数据的DataFrame
    all_df = pd.DataFrame(all_data, columns=["Model", "Temperature Value", "Accuracy"])

    # 对所有paper的数据进行平均计算
    final_avg_df = all_df.groupby(["Model", "Temperature Value"]).mean().reset_index()

    # 转换格式以方便展示
    final_pivot_df = final_avg_df.pivot(index="Model", columns="Temperature Value", values="Accuracy")
    final_pivot_df['Avg'] = final_pivot_df.mean(axis=1)
    final_pivot_df.loc['Avg'] = final_pivot_df.mean()
    final_pivot_df = final_pivot_df.sort_values(by='Avg', ascending=False)

    styled_final_df = final_pivot_df.style.apply(highlight_max_in_row, axis=None).apply(highlight_top_three, subset=final_pivot_df.columns.drop('Avg').tolist() + ['Avg'])
    styled_final_df = highlight_row_name(styled_final_df, 'Avg')
    styled_final_df = styled_final_df.set_caption(f'All Questions\' Accuracy: {paper_type}')
    styled_final_df = styled_final_df.format('{:.2%}')

    tables.append(styled_final_df)

    return tables


def display_multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )


In [16]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['llm_parameter']['temperature'] == temp:
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'ALL')
display_multi_table(tables)

Temperature Value,0.100000,0.300000,0.500000,0.700000,0.900000,Avg
Model,,,,,,
deepseek-coder,56.25%,54.91%,52.68%,53.57%,51.34%,53.75%
qwen2-72b-instruct,53.57%,54.91%,53.57%,48.21%,50.89%,52.23%
deepseek-chat,52.68%,52.23%,50.89%,51.34%,51.34%,51.70%
gemini-1.5-pro-001,46.43%,48.66%,50.00%,43.75%,48.21%,47.41%
gpt-4o-2024-05-13,45.09%,44.20%,44.64%,44.20%,41.52%,43.93%
claude-3-5-sonnet-20240620,43.30%,42.41%,40.18%,39.73%,44.64%,42.05%
baichuan4,41.52%,43.30%,40.18%,35.27%,38.39%,39.73%
yi-large,44.64%,46.88%,37.95%,37.05%,31.70%,39.64%
ERNIE-4.0-Turbo-8K,31.70%,31.25%,35.27%,35.27%,34.38%,33.57%


In [17]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['llm_parameter']['temperature'] == temp and i['prompt_parameter']['lang'] == 'en':
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'EN')
display_multi_table(tables)

Temperature Value,0.100000,0.300000,0.500000,0.700000,0.900000,Avg
Model,,,,,,
qwen2-72b-instruct,53.57%,51.79%,54.46%,48.21%,52.68%,52.14%
deepseek-chat,43.75%,46.43%,44.64%,42.86%,51.79%,45.89%
deepseek-coder,46.43%,45.54%,45.54%,43.75%,45.54%,45.36%
gemini-1.5-pro-001,42.86%,47.32%,46.43%,43.75%,46.43%,45.36%
gpt-4o-2024-05-13,45.54%,44.64%,44.64%,45.54%,42.86%,44.64%
claude-3-5-sonnet-20240620,43.75%,41.96%,39.29%,36.61%,43.75%,41.07%
baichuan4,42.86%,46.43%,41.96%,33.93%,37.50%,40.54%
yi-large,42.86%,45.54%,33.04%,34.82%,30.36%,37.32%
ERNIE-4.0-Turbo-8K,34.82%,31.25%,37.50%,35.71%,36.61%,35.18%


In [18]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['llm_parameter']['temperature'] == temp and i['prompt_parameter']['lang'] == 'zh':
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'ZH')
display_multi_table(tables)

Temperature Value,0.100000,0.300000,0.500000,0.700000,0.900000,Avg
Model,,,,,,
deepseek-coder,66.07%,64.29%,59.82%,63.39%,57.14%,62.14%
deepseek-chat,61.61%,58.04%,57.14%,59.82%,50.89%,57.50%
qwen2-72b-instruct,53.57%,58.04%,52.68%,48.21%,49.11%,52.32%
gemini-1.5-pro-001,50.00%,50.00%,53.57%,43.75%,50.00%,49.46%
gpt-4o-2024-05-13,44.64%,43.75%,44.64%,42.86%,40.18%,43.21%
claude-3-5-sonnet-20240620,42.86%,42.86%,41.07%,42.86%,45.54%,43.04%
yi-large,46.43%,48.21%,42.86%,39.29%,33.04%,41.97%
baichuan4,40.18%,40.18%,38.39%,36.61%,39.29%,38.93%
doubao-pro-32k-240615,37.50%,41.96%,35.71%,34.82%,36.61%,37.32%


## Print_question

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            bool:{
                "accuracy": ..,
                "avg_score": ..., 
                "total_score": ..., 
                "correct_cnt": ..., 
                "total_cnt": ...
            }
        }
    }
}
```

Print_question values: [True, False]

In [19]:
values = [True, False]

In [20]:
def generate_table(ans_data, papers, paper_type):

    def highlight_greater_between_col(row, col1, col2):
        # Initialize styles for each cell in the row as empty
        styles = [''] * len(row)
        # Determine the columns' positions
        col1_pos = row.index.get_loc(col1)
        col2_pos = row.index.get_loc(col2)
        # Apply style to the greater value between specified columns
        if row[col1] > row[col2]:
            styles[col1_pos] = 'text-decoration: underline'
        elif row[col1] < row[col2]:
            styles[col2_pos] = 'text-decoration: underline'
        return styles

    tables = []

    all_data = []

    for paper in papers:
        data = []
        for model_name, questions in ans_data[paper].items():
            for temp_value, metrics in questions.items():
                question_num = bool(temp_value)
                data.append([model_name, question_num, metrics["accuracy"]])

        df = pd.DataFrame(data, columns=["Model", "Print Question", "Accuracy"])
        pivot_df = df.pivot(index="Model", columns="Print Question", values="Accuracy").reindex(columns=[True, False])
        pivot_df['Avg'] = pivot_df.mean(axis=1)
        pivot_df = pivot_df.sort_values(by='Avg', ascending=False)
        pivot_df.loc['Avg'] = pivot_df.mean(axis=0)

        styled_df = pivot_df.style.apply(highlight_top_three, subset=pivot_df.columns.tolist())
        styled_df = styled_df.apply(highlight_greater_between_col, col1=True, col2=False, axis=1)
        styled_df = styled_df.set_caption(f'{paper} Question Accuracy: {paper_type}')
        styled_df = styled_df.format('{:.2%}')
        tables.append(styled_df)
        
        for index, row in pivot_df.iterrows():
            for col in pivot_df.columns.drop('Avg'):
                all_data.append([index, col, row[col]])

    all_df = pd.DataFrame(all_data, columns=["Model", "Print Question", "Accuracy"])
    final_avg_df = all_df.groupby(["Model", "Print Question"]).mean().reset_index()
    final_pivot_df = final_avg_df.pivot(index="Model", columns="Print Question", values="Accuracy").reindex(columns=[True, False])
    final_pivot_df['Avg'] = final_pivot_df.mean(axis=1)
    final_pivot_df = final_pivot_df.sort_values(by='Avg', ascending=False)
    final_pivot_df.loc['Avg'] = final_pivot_df.mean(axis=0)

    styled_final_df = final_pivot_df.style.apply(highlight_top_three, subset=final_pivot_df.columns.tolist())
    styled_final_df = highlight_row_name(styled_final_df, 'Avg')
    styled_final_df = styled_final_df.apply(highlight_greater_between_col, col1=True, col2=False, axis=1)
    styled_final_df = styled_final_df.set_caption(f'All Questions\' Accuracy: {paper_type}')
    styled_final_df = styled_final_df.format('{:.2%}')

    tables.append(styled_final_df)

    return tables



def display_multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

In [21]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['print_question'] == temp:
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'Print Question')
display_multi_table(tables)

Print Question,True,False,Avg
Model,,,
deepseek-coder,54.64%,52.86%,53.75%
qwen2-72b-instruct,51.79%,52.68%,52.24%
deepseek-chat,51.25%,52.14%,51.70%
gemini-1.5-pro-001,49.82%,45.00%,47.41%
gpt-4o-2024-05-13,42.86%,45.00%,43.93%
claude-3-5-sonnet-20240620,42.14%,41.96%,42.05%
baichuan4,39.82%,39.64%,39.73%
yi-large,39.64%,39.64%,39.64%
ERNIE-4.0-Turbo-8K,37.14%,30.00%,33.57%


## CoT

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            bool:{
                "accuracy": ..,
                "avg_score": ..., 
                "total_score": ..., 
                "correct_cnt": ..., 
                "total_cnt": ...
            }
        }
    }
}
```

CoT values: [True, False]

In [23]:
def generate_table(ans_data, papers, paper_type):

    def highlight_greater_between_col(row, col1, col2):
        # Initialize styles for each cell in the row as empty
        styles = [''] * len(row)
        # Determine the columns' positions
        col1_pos = row.index.get_loc(col1)
        col2_pos = row.index.get_loc(col2)
        # Apply style to the greater value between specified columns
        if row[col1] > row[col2]:
            styles[col1_pos] = 'text-decoration: underline'
        elif row[col1] < row[col2]:
            styles[col2_pos] = 'text-decoration: underline'
        return styles

    tables = []

    all_data = []

    for paper in papers:
        data = []
        for model_name, questions in ans_data[paper].items():
            for temp_value, metrics in questions.items():
                question_num = bool(temp_value)
                data.append([model_name, question_num, metrics["accuracy"]])

        df = pd.DataFrame(data, columns=["Model", "CoT", "Accuracy"])
        pivot_df = df.pivot(index="Model", columns="CoT", values="Accuracy").reindex(columns=[True, False])
        pivot_df['Avg'] = pivot_df.mean(axis=1)
        pivot_df = pivot_df.sort_values(by='Avg', ascending=False)
        pivot_df.loc['Avg'] = pivot_df.mean(axis=0)

        styled_df = pivot_df.style.apply(highlight_top_three, subset=pivot_df.columns.tolist())
        styled_df = styled_df.apply(highlight_greater_between_col, col1=True, col2=False, axis=1)
        styled_df = styled_df.set_caption(f'{paper} Question Accuracy: {paper_type}')
        styled_df = styled_df.format('{:.2%}')
        tables.append(styled_df)
        
        for index, row in pivot_df.iterrows():
            for col in pivot_df.columns.drop('Avg'):
                all_data.append([index, col, row[col]])

    all_df = pd.DataFrame(all_data, columns=["Model", "CoT", "Accuracy"])
    final_avg_df = all_df.groupby(["Model", "CoT"]).mean().reset_index()
    final_pivot_df = final_avg_df.pivot(index="Model", columns="CoT", values="Accuracy").reindex(columns=[True, False])
    final_pivot_df['Avg'] = final_pivot_df.mean(axis=1)
    final_pivot_df = final_pivot_df.sort_values(by='Avg', ascending=False)

    styled_final_df = final_pivot_df.style.apply(highlight_top_three, subset=final_pivot_df.columns.tolist())
    styled_final_df = highlight_row_name(styled_final_df, 'Avg')
    styled_final_df = styled_final_df.apply(highlight_greater_between_col, col1=True, col2=False, axis=1)
    styled_final_df = styled_final_df.set_caption(f'All Questions\' Accuracy: {paper_type}')
    styled_final_df = styled_final_df.format('{:.2%}')

    tables.append(styled_final_df)

    return tables

def display_multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

In [24]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['cot'] == temp:
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'CoT')
display_multi_table(tables)

CoT,True,False,Avg
Model,,,
deepseek-coder,53.93%,53.57%,53.75%
qwen2-72b-instruct,51.96%,52.50%,52.23%
deepseek-chat,51.07%,52.32%,51.70%
gemini-1.5-pro-001,48.39%,46.43%,47.41%
gpt-4o-2024-05-13,44.64%,43.21%,43.93%
claude-3-5-sonnet-20240620,44.29%,39.82%,42.05%
baichuan4,40.36%,39.11%,39.73%
yi-large,40.36%,38.93%,39.64%
ERNIE-4.0-Turbo-8K,34.29%,32.86%,33.58%


In [25]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['cot'] == temp and i['prompt_parameter']['lang'] == 'en':
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'CoT-EN')
display_multi_table(tables)

CoT,True,False,Avg
Model,,,
qwen2-72b-instruct,53.93%,50.36%,52.14%
deepseek-chat,44.64%,47.14%,45.89%
deepseek-coder,45.36%,45.36%,45.36%
gemini-1.5-pro-001,46.79%,43.93%,45.36%
gpt-4o-2024-05-13,45.00%,44.29%,44.65%
claude-3-5-sonnet-20240620,43.21%,38.93%,41.07%
baichuan4,41.07%,40.00%,40.53%
yi-large,40.71%,33.93%,37.32%
ERNIE-4.0-Turbo-8K,35.71%,34.64%,35.17%


In [26]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['cot'] == temp and i['prompt_parameter']['lang'] == 'zh':
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'CoT-ZH')
display_multi_table(tables)

CoT,True,False,Avg
Model,,,
deepseek-coder,62.50%,61.79%,62.15%
deepseek-chat,57.50%,57.50%,57.50%
qwen2-72b-instruct,50.00%,54.64%,52.32%
gemini-1.5-pro-001,50.00%,48.93%,49.47%
gpt-4o-2024-05-13,44.29%,42.14%,43.22%
claude-3-5-sonnet-20240620,45.36%,40.71%,43.04%
yi-large,40.00%,43.93%,41.97%
baichuan4,39.64%,38.21%,38.92%
doubao-pro-32k-240615,36.07%,38.57%,37.32%


## Sequential

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            'answer-first':{
                "accuracy": ..,
                "avg_score": ..., 
                "total_score": ..., 
                "correct_cnt": ..., 
                "total_cnt": ...
            }
        }
    }
}
```

Sequential values: ['answer-first', 'process-first']

In [27]:
def generate_table(ans_data, papers, paper_type):

    def highlight_greater_between_col(row, col1, col2):
        # Initialize styles for each cell in the row as empty
        styles = [''] * len(row)
        # Determine the columns' positions
        col1_pos = row.index.get_loc(col1)
        col2_pos = row.index.get_loc(col2)
        # Apply style to the greater value between specified columns
        if row[col1] > row[col2]:
            styles[col1_pos] = 'text-decoration: underline'
        elif row[col1] < row[col2]:
            styles[col2_pos] = 'text-decoration: underline'
        return styles

    tables = []

    all_data = []

    for paper in papers:
        data = []
        for model_name, questions in ans_data[paper].items():
            for temp_value, metrics in questions.items():
                question_num = temp_value
                data.append([model_name, question_num, metrics["accuracy"]])

        df = pd.DataFrame(data, columns=["Model", "Sequential", "Accuracy"])
        pivot_df = df.pivot(index="Model", columns="Sequential", values="Accuracy")
        pivot_df['Avg'] = pivot_df.mean(axis=1)
        pivot_df = pivot_df.sort_values(by='Avg', ascending=False)
        pivot_df.loc['Avg'] = pivot_df.mean(axis=0)

        styled_df = pivot_df.style.apply(highlight_top_three, subset=pivot_df.columns.tolist())
        styled_df = styled_df.apply(highlight_greater_between_col, col1='answer-first', col2='process-first', axis=1)
        styled_df = styled_df.set_caption(f'{paper} Question Accuracy: {paper_type}')
        styled_df = styled_df.format('{:.2%}')
        tables.append(styled_df)
        
        for index, row in pivot_df.iterrows():
            for col in pivot_df.columns.drop('Avg'):
                all_data.append([index, col, row[col]])

    all_df = pd.DataFrame(all_data, columns=["Model", "Sequential", "Accuracy"])
    final_avg_df = all_df.groupby(["Model", "Sequential"]).mean().reset_index()
    final_pivot_df = final_avg_df.pivot(index="Model", columns="Sequential", values="Accuracy")
    final_pivot_df['Avg'] = final_pivot_df.mean(axis=1)
    final_pivot_df = final_pivot_df.sort_values(by='Avg', ascending=False)

    styled_final_df = final_pivot_df.style.apply(highlight_top_three, subset=final_pivot_df.columns.tolist())
    styled_final_df = highlight_row_name(styled_final_df, 'Avg')
    styled_final_df = styled_final_df.apply(highlight_greater_between_col, col1='answer-first', col2='process-first', axis=1)
    styled_final_df = styled_final_df.set_caption(f'All Questions\' Accuracy: {paper_type}')
    styled_final_df = styled_final_df.format('{:.2%}')

    tables.append(styled_final_df)

    return tables

def display_multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

In [28]:
values = ['answer-first', 'process-first']

In [29]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['sequential'] == temp:
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'Sequential')
display_multi_table(tables)

Sequential,answer-first,process-first,Avg
Model,,,
deepseek-coder,42.50%,65.00%,53.75%
qwen2-72b-instruct,48.75%,55.71%,52.23%
deepseek-chat,41.79%,61.61%,51.70%
gemini-1.5-pro-001,43.93%,50.89%,47.41%
gpt-4o-2024-05-13,33.57%,54.29%,43.93%
claude-3-5-sonnet-20240620,26.25%,57.86%,42.05%
baichuan4,40.00%,39.46%,39.73%
yi-large,40.18%,39.11%,39.64%
ERNIE-4.0-Turbo-8K,22.14%,45.00%,33.57%


In [30]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['sequential'] == temp and i['prompt_parameter']['lang'] == 'en':
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'Sequential-EN')
display_multi_table(tables)

Sequential,answer-first,process-first,Avg
Model,,,
qwen2-72b-instruct,48.93%,55.36%,52.14%
deepseek-chat,39.29%,52.50%,45.90%
deepseek-coder,33.93%,56.79%,45.36%
gemini-1.5-pro-001,40.36%,50.36%,45.36%
gpt-4o-2024-05-13,33.93%,55.36%,44.65%
claude-3-5-sonnet-20240620,25.71%,56.43%,41.07%
baichuan4,44.29%,36.79%,40.54%
yi-large,37.14%,37.50%,37.32%
ERNIE-4.0-Turbo-8K,25.71%,44.64%,35.17%


In [31]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for temp in values:

            ans_data[paper][model][temp] = {}

            total_cnt, correct_cnt, total_score = 0, 0, 0

            for fpath in fpaths:
                
                content = parse_data.read_jsonl(fpath)

                for i in content:
                    if i['prompt_parameter']['sequential'] == temp and i['prompt_parameter']['lang'] == 'zh':
                        total_cnt += 1
                        if i['answer']['correct']:
                            correct_cnt += 1
                        total_score += i['answer']['score']

            ans_data[paper][model][temp]['total_cnt'] = total_cnt
            ans_data[paper][model][temp]['correct_cnt'] = correct_cnt
            ans_data[paper][model][temp]['total_score'] = total_score
            ans_data[paper][model][temp]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][temp]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_table(ans_data, papers, 'Sequential-ZH')
display_multi_table(tables)

Sequential,answer-first,process-first,Avg
Model,,,
deepseek-coder,51.07%,73.21%,62.14%
deepseek-chat,44.29%,70.71%,57.50%
qwen2-72b-instruct,48.57%,56.07%,52.32%
gemini-1.5-pro-001,47.50%,51.43%,49.46%
gpt-4o-2024-05-13,33.21%,53.21%,43.21%
claude-3-5-sonnet-20240620,26.79%,59.29%,43.04%
yi-large,43.21%,40.71%,41.96%
baichuan4,35.71%,42.14%,38.92%
doubao-pro-32k-240615,20.00%,54.64%,37.32%


## Token Number

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            'question_idx':{
                "correct_cnt": ..., 
                "incorrect_cnt": ....,
                "total_cnt": ...,
                "correct_token": ...,
                "incorrect_token": ...,
                "total_token": ...,
                "avg_correct_tokens": ...,
                "avg_incorrect_tokens": ...,
                "avg_tokens": ...,
            }
        }
    }
}
```

In [32]:
import pandas as pd

def generate_tables(ans_data, papers):
    tables = []

    for paper in papers:
        data = []
        # Ensure iterating over all models for the specified paper
        for model_name, questions in ans_data[paper].items():
            for question_idx, metrics in questions.items():
                question_num = int(question_idx.replace('question', ''))
                # Add data for each metric type under each question
                data.extend([
                    (model_name, question_num, 'True', metrics['avg_correct_tokens']),
                    (model_name, question_num, 'False', metrics['avg_incorrect_tokens']),
                    (model_name, question_num, 'Total', metrics['avg_tokens'])
                ])

        # Create DataFrame
        df = pd.DataFrame(data, columns=["Model", "Question Index", "Metric Type", "Value"])

        # Pivot the DataFrame to get the desired format with multi-level columns
        pivot_df = df.pivot_table(
            index="Model",
            columns=["Question Index", "Metric Type"],
            values="Value",
            aggfunc='first'
        )

        # Reorder the levels to sort by Metric Type within each Question Index
        metric_order = ["True", "False", "Total"]  # Define the desired order for Metric Type
        # Reindex the columns based on the new order for each Question Index
        new_columns = pd.MultiIndex.from_product([pivot_df.columns.levels[0], metric_order], names=pivot_df.columns.names)
        pivot_df = pivot_df.reindex(columns=new_columns)

        # Sort the index to ensure models are in alphabetical order
        pivot_df = pivot_df.sort_index()

        # Set the caption for the DataFrame
        styled_df = pivot_df.style.set_caption(f'{paper} Token Metrics')

        # Format the numbers to no decimal places
        styled_df = styled_df.format('{:.0f}')

        # Collect all tables for return
        tables.append(styled_df)

    return tables

In [33]:
ans_data = {}
tiny_number = 1e-10

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, incorrect_cnt = 0, 0, 0
            total_tokens, correct_tokens, incorrect_tokens = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                # ignore the missing data
                if 'total_tokens' not in i['token_usage'].keys():
                    continue

                total_cnt += 1
                if i['answer']['correct']:
                    correct_cnt += 1
                    correct_tokens += i['token_usage']['total_tokens']
                else:
                    incorrect_cnt += 1
                    incorrect_tokens += i['token_usage']['total_tokens']

            if correct_cnt == 0:
                correct_cnt = tiny_number
            if incorrect_cnt == 0:
                incorrect_cnt = tiny_number
            if total_cnt == 0:
                total_cnt = tiny_number

            total_tokens = incorrect_tokens + correct_tokens
            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['incorrect_cnt'] = incorrect_cnt
            ans_data[paper][model][ques_idx]['correct_tokens'] = correct_tokens
            ans_data[paper][model][ques_idx]['incorrect_tokens'] = incorrect_tokens
            ans_data[paper][model][ques_idx]['total_tokens'] = total_tokens
            ans_data[paper][model][ques_idx]['avg_correct_tokens'] = correct_tokens / correct_cnt
            ans_data[paper][model][ques_idx]['avg_incorrect_tokens'] = incorrect_tokens / incorrect_cnt
            ans_data[paper][model][ques_idx]['avg_tokens'] = total_tokens / total_cnt


tables = generate_tables(ans_data, papers)
display(tables[0])
display(tables[1])

# Score

```json
ans_data = {
    "paper_type": 
    {
        "model_name": {
            "question_idx":{
                "accuracy": ..,
                "avg_score": ..., 
                "total_score": ..., 
                "correct_cnt": ..., 
                "total_cnt": ...
            }
        }
    }
}
```

In [34]:
def generate_acc_table(ans_data, papers, paper_type):

    tables = []

    for paper in papers:
        data = []
        for model_name, questions in ans_data[paper].items():
            for question_idx, metrics in questions.items():
                question_num = int(question_idx.replace('question', ''))
                data.append([model_name, question_num, metrics["avg_score"]])

        df = pd.DataFrame(data, columns=["Model", "Question Index", "Score"])

        # Pivot table to get the desired format
        pivot_df = df.pivot(index="Model", columns="Question Index", values="Score")

        # Calculate the average accuracy for each model and add as a new column
        pivot_df['Total'] = pivot_df.sum(axis=1)
        pivot_df['Rate'] = pivot_df['Total'] /73

        pivot_df.loc['Avg'] = pivot_df.mean(axis=0)

        # Sort by Avg column in descending order
        pivot_df = pivot_df.sort_values(by='Total', ascending=False)

        # Apply the highlighting function
        styled_df = pivot_df.style.apply(highlight_top_three, subset=[i for i in range(1, 15)] + ['Total'])
        styled_df = highlight_row_name(styled_df, 'Avg')

        # Set the caption for the DataFrame
        styled_df = styled_df.set_caption(f'{paper} Question Score: {paper_type}')

        styled_df = styled_df.format({
            'Rate': '{:.2%}',   # 为 'Rate' 列应用百分比格式，并保留四位小数
            **{col: '{:.2f}' for col in pivot_df.columns if col != 'Rate'}  # 为除 'Rate' 外的其它列应用浮点数格式，并保留两位小数
        })
        

        # Display the styled DataFrame
        tables.append(styled_df)

    return tables

In [35]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, total_score = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                total_cnt += 1
                if i['answer']['correct']:
                    correct_cnt += 1
                total_score += i['answer']['score']

            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['total_score'] = total_score
            ans_data[paper][model][ques_idx]['avg_score'] = total_score/total_cnt
            ans_data[paper][model][ques_idx]['accuracy'] = correct_cnt/total_cnt

tables = generate_acc_table(ans_data, papers, 'ALL')
display_multi_table(tables)

Question Index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Total,Rate
Model,,,,,,,,,,,,,,,,
deepseek-coder,4.94,1.62,2.50,2.62,4.56,3.81,5.00,3.19,3.90,1.46,3.60,1.44,2.25,0.06,40.96,56.11%
qwen2-72b-instruct,4.94,2.88,2.94,1.19,4.88,4.75,4.31,2.81,4.28,1.12,3.94,0.25,0.94,1.00,40.21,55.09%
deepseek-chat,5.00,1.38,2.50,2.62,4.94,4.31,5.00,1.94,3.60,1.73,2.85,1.12,1.75,0.12,38.86,53.24%
gemini-1.5-pro-001,1.94,2.12,4.75,1.31,5.00,3.88,4.50,3.12,3.83,1.65,1.76,0.38,1.62,0.12,35.99,49.30%
gpt-4o-2024-05-13,4.88,3.19,2.81,1.00,3.25,2.88,4.62,0.19,2.55,0.11,3.86,0.75,1.81,0.38,32.28,44.21%
claude-3-5-sonnet-20240620,2.19,4.69,4.06,1.44,3.75,4.75,2.81,0.94,2.55,0.94,0.82,0.50,1.31,0.19,30.94,42.38%
baichuan4,2.88,2.62,3.50,1.69,3.00,3.50,3.38,2.00,2.51,0.45,2.55,0.00,0.69,1.75,30.51,41.80%
yi-large,2.81,3.00,3.75,1.62,3.12,2.62,3.38,2.12,2.74,0.34,2.70,0.00,0.94,0.69,29.84,40.87%
Avg,2.96,2.27,3.05,1.21,3.41,3.57,3.74,1.61,2.36,0.88,2.49,0.40,0.84,0.43,29.24,40.05%


In [36]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, total_score = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                if i['prompt_parameter']['lang'] == 'zh':
                    total_cnt += 1
                    if i['answer']['correct']:
                        correct_cnt += 1
                    total_score += i['answer']['score']

            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['total_score'] = total_score
            ans_data[paper][model][ques_idx]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][ques_idx]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_acc_table(ans_data, papers, 'ZH')
display_multi_table(tables)

Question Index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Total,Rate
Model,,,,,,,,,,,,,,,,
deepseek-coder,4.88,2.25,2.50,2.88,4.62,4.75,5.00,4.25,5.85,2.70,3.67,1.75,2.00,0.00,47.10,64.52%
deepseek-chat,5.00,2.12,2.50,3.00,4.88,5.00,5.00,2.25,4.50,2.55,2.02,2.00,2.00,0.00,42.82,58.66%
qwen2-72b-instruct,4.88,4.12,2.50,1.12,4.75,4.88,4.75,1.75,4.35,1.65,3.23,0.12,1.88,1.12,41.10,56.30%
gemini-1.5-pro-001,2.00,2.00,4.88,1.25,5.00,4.25,4.75,3.00,4.65,1.65,1.50,0.62,1.62,0.12,37.30,51.10%
gpt-4o-2024-05-13,4.75,3.62,2.88,0.75,2.88,3.00,4.38,0.38,2.70,0.23,3.75,0.50,1.88,0.25,31.93,43.73%
claude-3-5-sonnet-20240620,2.50,4.62,3.62,1.62,3.75,4.50,4.25,1.00,1.65,0.45,0.68,0.88,1.25,0.38,31.15,42.67%
yi-large,3.12,3.00,3.75,1.62,3.38,3.12,4.00,2.50,2.17,0.07,2.33,0.00,1.12,0.88,31.07,42.57%
Avg,2.91,2.51,3.12,1.26,3.30,3.86,3.91,1.70,2.62,1.00,2.32,0.59,0.90,0.46,30.46,41.73%
baichuan4,2.50,2.62,3.38,1.88,1.75,3.38,4.38,2.38,2.33,0.53,2.77,0.00,0.62,1.62,30.12,41.27%


In [37]:
ans_data = {}

for paper in papers:
    ans_data[paper] = {}

    for model in models:

        ans_data[paper][model] = {}
        model_res_dir = os.path.join(res_dir, model)
        paper_res_dir = os.path.join(model_res_dir, paper)
        fpaths = parse_data.find_files_with_suffix(paper_res_dir, '.jsonl')

        for fpath in fpaths:
            
            ques_idx = fpath.split(os.sep)[-1].split('.')[0]
            ans_data[paper][model][ques_idx] = {}
            total_cnt, correct_cnt, total_score = 0, 0, 0

            content = parse_data.read_jsonl(fpath)

            for i in content:
                if i['prompt_parameter']['lang'] == 'en':
                    total_cnt += 1
                    if i['answer']['correct']:
                        correct_cnt += 1
                    total_score += i['answer']['score']

            ans_data[paper][model][ques_idx]['total_cnt'] = total_cnt
            ans_data[paper][model][ques_idx]['correct_cnt'] = correct_cnt
            ans_data[paper][model][ques_idx]['total_score'] = total_score
            ans_data[paper][model][ques_idx]['avg_score'] = round(total_score/total_cnt, 4)
            ans_data[paper][model][ques_idx]['accuracy'] = round(correct_cnt/total_cnt, 4)

tables = generate_acc_table(ans_data, papers, 'EN')
display_multi_table(tables)

Question Index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Total,Rate
Model,,,,,,,,,,,,,,,,
qwen2-72b-instruct,5.00,1.62,3.38,1.25,5.00,4.62,3.88,3.88,4.20,0.60,4.65,0.38,0.00,0.88,39.33,53.87%
deepseek-chat,5.00,0.62,2.50,2.25,5.00,3.62,5.00,1.62,2.70,0.90,3.67,0.25,1.50,0.25,34.90,47.81%
deepseek-coder,5.00,1.00,2.50,2.38,4.50,2.88,5.00,2.12,1.95,0.23,3.52,1.12,2.50,0.12,34.83,47.71%
gemini-1.5-pro-001,1.88,2.25,4.62,1.38,5.00,3.50,4.25,3.25,3.00,1.65,2.02,0.12,1.62,0.12,34.67,47.50%
gpt-4o-2024-05-13,5.00,2.75,2.75,1.25,3.62,2.75,4.88,0.00,2.40,0.00,3.98,1.00,1.75,0.50,32.62,44.69%
baichuan4,3.25,2.62,3.62,1.50,4.25,3.62,2.38,1.62,2.70,0.38,2.33,0.00,0.75,1.88,30.90,42.33%
claude-3-5-sonnet-20240620,1.88,4.75,4.50,1.25,3.75,5.00,1.38,0.88,3.45,1.43,0.97,0.12,1.38,0.00,30.73,42.09%
yi-large,2.50,3.00,3.75,1.62,2.88,2.12,2.75,1.75,3.30,0.60,3.08,0.00,0.75,0.50,28.60,39.18%
Avg,3.02,2.04,2.99,1.16,3.51,3.28,3.58,1.52,2.10,0.75,2.67,0.21,0.78,0.39,28.01,38.37%


# Repetition

In [57]:
res = dict()

for model in models:
    model_res_dir = os.path.join(res_dir, model)
    fpaths = parse_data.find_files_with_suffix(model_res_dir, '.jsonl')
    res[model] = Counter()

    for fpath in fpaths:
        content = parse_data.read_jsonl(fpath)

        for i in content:
            if 'repetition' in i['answer'].keys():
                res[model]['bad_case'] += 1
                if i['answer']['repetition']:
                    res[model]['repetition'] += 1
                else:
                    res[model]['repetition'] += 0

# Convert the counter to a DataFrame
data = [{'Model': model, 'Bad Cases': counts['bad_case'], 'Repetitions': counts['repetition']} for model, counts in res.items()]
df = pd.DataFrame(data)

# Calculate Repetition Rate and format it as a percentage with two decimal places
df['Repetition Rate'] = df['Repetitions'] / df['Bad Cases']
df['Repetition Rate'] = df['Repetition Rate'].fillna(0)  # Fill NaN values with zero if there are no bad cases
df['Repetition Rate'] = df['Repetition Rate'].apply(lambda x: "{:.2%}".format(x))

# Sort the DataFrame by 'Repetition Rate' in ascending order
df.sort_values(by='Repetition Rate', ascending=True, na_position='first', inplace=True)

# Rearrange columns and reset index
df = df[['Model', 'Repetitions', 'Bad Cases', 'Repetition Rate']]
df.reset_index(drop=True, inplace=True)

df

,Model,Repetitions,Bad Cases,Repetition Rate
0,claude-3-5-sonnet-20240620,0,0,0.00%
1,doubao-pro-32k-240615,0,2,0.00%
2,gemini-1.5-pro-001,1,8,12.50%
3,mistral-large-2402,7,34,20.59%
4,ERNIE-4.0-Turbo-8K,124,597,20.77%
5,deepseek-chat,12,48,25.00%
6,deepseek-coder,14,55,25.45%
7,baichuan4,126,393,32.06%
8,qwen2-72b-instruct,2,6,33.33%
9,gpt-4o-2024-05-13,17,45,37.78%
